# to run in all cases
### get the credentials and instantiate the fb_client

### Environmental Variables based on App and User/Page
App M001
 - FB_M001_App_id
 - FB_M001_App_secret
 - FB_M001_user_token
 - FB_M001_DH_App_token (Access toke page)
 - FB_M001_ES_App_token (Access toke page)

ES
 - FB_ES_App_id
 - FB_ES_App_secret
 - FB_ES_user_token
 - FB_ES_ES_App_token (Access toke page)


Pages:
 - FB_DH_Pg_id - DH Page ID: 122097693698010058 
 - FB_ES_Pg_id - ES Page ID: 303403436189223


In [ ]:
import os
from typing import Dict, Optional, Any
import json
import facebook
from facebook_api_client import FacebookAPIClient
from post_manager import PostManager
from utils import Utils

# Load credentials (RELEVANT IF I AM GOING TO USE
# DIFFERENT CREDENTIALS THAT THOSE DEFINED ON THE FacebookAPIClient Class load credentials)

try:
    app_id = os.environ["FB_M001_App_id"]
    app_secret = os.environ["FB_M001_App_secret"]

    page_id = os.environ["FB_ES_Pg_id"]
    access_token = os.environ["FB_M001_ES_App_token"]
    
except KeyError:
    print("Error: Environment variables not set. Please set them before running the tests.")
    exit(1)  # Exit with error code

print(app_id, app_secret, page_id, access_token)
# Instantiate the FacebookAPIClient and PostManager
# fb_client = FacebookAPIClient(app_id, app_secret, access_token, page_id)
# post_manager = PostManager(fb_client)
# utils = Utils(fb_client)

In [ ]:
print(dir(post_manager)) 
print(dir(Utils))


## Test to get information about a Group
### when only the name is known


# PENDING CHECK

In [ ]:
def get_group_id_by_name(api_client: facebook.GraphAPI, group_name: str) -> Optional[str]:
    """Retrieves the ID of a group by its name.

    Args:
        api_client: A Facebook GraphAPI instance.
        group_name: The name of the group.

    Returns:
        The group ID if found, None otherwise.
    """
    try:
        groups = api_client.get_connections(id="me", connection_name="groups")
        for group in groups["data"]:
            if group["name"] == group_name:
                return group["id"]
    except facebook.GraphAPIError as e:
        print(f"Error retrieving groups: {e.message}")
    return None

# ... (your existing code)

group_id = get_group_id_by_name(fb_client, "YOUR_GROUP_NAME")
if group_id:
    group_info = Utils.get_group_info(fb_client, group_id)
    # ... do something with the group_info
else:
    print("Group not found.")


### when the id is known

In [ ]:
group_id = "490852652771054"
group_data = utils.get_group_info(fb_client, group_id)
print(group_data) 

## Test to get all the Groups followed by a User or a Page, provided the user or page id

In [ ]:
# Let's try to check at least my own page groups with a direct code

import facebook

# Use the GraphAPI object from your FacebookAPIClient
graph = fb_client.get_graph_api_object()

# Get the user ID associated with your access token
me = graph.get_object(id="me")
user_id = me['id']
print(user_id)

# Get the groups followed by the user, filtering to include only the page
groups_data = graph.get_connections(
    id=user_id, 
    connection_name="groups", 
    fields="id,name,description"
)

# Filter the groups to find those where the page is a member
page_groups = [
    group 
    for group in groups_data["data"] 
    if graph.get_object(id=group['id'], fields="id").get('id') == page_id
]

# If any groups are found, print them
if page_groups:
    for group in page_groups:
        print(f"Group ID: {group['id']}, Name: {group['name']}, Description: {group['description']}")
else:
    print("The page does not follow any groups.")


In [ ]:
# Get groups followed by the user

user_or_page_id = page_id
print(user_or_page_id)
groups = utils.get_groups_followed(user_or_page_id)
      
if groups:
    for group in groups:
        print(f"Group ID: {group['id']}, Name: {group['name']}, Description: {group['description']}")
else:
    print(
        "No groups found. This may be due to missing permissions or the user/page not being a member of any groups."
    )

## Test to grab the latest post

In [ ]:
try:
    latest_post = post_manager.get_latest_post(page_id)
    if latest_post:
        print("Latest post retrieved successfully:")
        print(f"  ID: {latest_post['id']}")
        print(f"  Message: {latest_post['message']}")
        print(f"  Post link: {latest_post['permalink_url']}")
        print(f"  Likes Count: {latest_post['likes']['summary']['total_count']}")
        print(f"  Comments Count: {latest_post['comments']['summary']['total_count']}")
        print("---------------------------------------------------------------------")
        
        # You can print other fields here as well
        import json
        print("latest post retrieved successfully:")
        print(json.dumps(latest_post, indent=4))      # Pretty-print the dictionary

    else:
        print("No latest post found or error retrieving post.")
except Exception as e:  # Catching general exceptions for this example
    print(f"Error retrieving latest post: {e}")

print(f" Post ID: {latest_post['id']}") 

## Test to grab the latests posts (num_posts)

In [ ]:
try:
    latest_posts = post_manager.get_latest_posts(page_id, num_posts=3) 
    if latest_posts:  # Check if the list is not empty
        print("Latest posts retrieved successfully:")
        for post in latest_posts:
            print(f"  ID: {post['id']}")
            print(f"  Message: {post.get('message', 'No message available')}")  # Use get() to handle missing messages
            print(f"  Post link: {post['permalink_url']}")
            print(f"  Likes Count: {post['likes']['summary']['total_count']}")
            print(f"  Comments Count: {post['comments']['summary']['total_count']}")
            print("---------------------------------------------------------------------")

            # You can print other fields here as well
            import json
            print(json.dumps(post, indent=4))      # Pretty-print each post individually

    else:
        print("No latest posts found or error retrieving posts.")
except Exception as e:  # Catching general exceptions for this example
    print(f"Error retrieving latest posts: {e}")



In [ ]:
# tests/test_post_manager.py (create a separate tests folder)
import unittest
from unittest.mock import patch  # Mocking for API calls
from post_manager import PostManager  # Assuming this is your module name

class TestPostManager(unittest.TestCase):
    def setUp(self):
        # Create a mock FacebookAPIClient
        self.mock_api_client = patch("post_manager.FacebookAPIClient").start()  # Mocking the api client for test
        self.post_manager = PostManager(self.mock_api_client)

    def tearDown(self):
        patch.stopall()

    @patch("facebook.GraphAPI.get_connections")  # Mock the get_connections method of the GraphAPI
    def test_get_latest_posts(self, mock_get_connections):
        # Mock API response data
        mock_response = {
            "data": [
                {
                    "id": "1234567890",
                    "message": "This is a test post.",
                    "created_time": "2024-05-23T12:00:00+0000",
                    "permalink_url": "https://www.facebook.com/1234567890",
                    "likes": {"summary": {"total_count": 10}},
                    "comments": {"summary": {"total_count": 5}},
                },
                # Add more mock post data as needed
            ]
        }
        mock_get_connections.return_value = mock_response

        posts = self.post_manager.get_latest_posts(page_id, num_posts=2)  # Request 2 posts
        # Printing results
        print("TEST OUTPUT - get_latest_posts")
        for post in posts:
            print("Post ID:", post['id'])
            print("Message:", post['message'])
            print("Likes:", post['likes']['summary']['total_count'])
            print("Comments:", post['comments']['summary']['total_count'])
            print("---")
        self.assertEqual(len(posts), 2)  # Check that the correct number of posts were returned
        
        # Check a few specific fields in the first post (customize these)
        self.assertEqual(posts[0]["id"], "1234567890")
        self.assertEqual(posts[0]["message"], "This is a test post.")

if __name__ == "__main__":
    %run post_manager.py


## Test to retrieve a post and its media by post id

In [ ]:

post_id = "303403436189223_122111155826316036"  # Replace with the actual post ID
specific_post = post_manager.get_post_by_id(post_id)
if specific_post:
    import json
    print(json.dumps(specific_post, indent=4)) 

    if "attachments" in specific_post:
        attachments = specific_post["attachments"]["data"]
        for attachment in attachments:
            media_type = attachment.get("media_type")
            if media_type == "photo":
                photo_url = attachment["media"]["image"]["src"]
                print(f"Found photo attachment: {photo_url}")
            elif media_type == "video":
                video_url = attachment["media"]["source"]
                print(f"Found video attachment: {video_url}")
            # Add other media types (e.g., "link", "album") if needed
    else:
        print("No attachments found in this post.")
else:
    print("No post found or error retrieving post.")


## Test to publish a message post

In [ ]:
   


# Publish a text post
text_post_message = "I loved 2024 Romeria de El Rocio"
try:
    text_post_result = post_manager.publish_text_post(page_id, text_post_message)
except Exception as e:  # Catching general exceptions for this example
    print(f"Error publishing text post: {e}")

print("Text post result:", text_post_result)


## Test to publish a one photo post with a message

In [ ]:


photo_post_message = "El Rocio so full with horses"
photo_path = r"C:\Users\manue\Documents\Disfrutando Huelva\Agrupados para editar\Rocio - Todos\Caballos carros y carretas en el rocio\20240518_131042~2.jpg"
try:
    photo_post_result = post_manager.publish_photo_post(page_id, photo_post_message, photo_path)
except Exception as e:  # Catching general exceptions for this example
    print(f"Error publishing photo post: {e}")

# Print results
    
print("Photo post result:", photo_post_result)
